# House Prices — Kernel Methods & Tree Models

_Updates: Made some parts of the code neater._

Below is a summary of the approach I have taken in this notebook.

**Data Preprocessing:**
- This section mainly deals with filling the missing values in the data. A number of columns have categories NA/None to denote that the corresponding feature of the house does not exist. It could be possible that the feature exists, but there are actual missing data values. To differentiate between the two cases, I give an example using the basement feature. If its corresponding columns `BsmtQual`, `BsmtFinSF1`, `BsmtFullBath`, etc. are all filled with 0 or NA, I assume that the particular house does not have a basement, and fill using the value 0 or string "NA" accordingly.
- After the first round of imputation, we still have missing values for the columns filled from the process described above, indicating that the data values are indeed missing. We fill the remaining values using some form of estimation. In particular, the log values of `LotFrontage` and `LotArea` are strongly correlated, so it is reasonable to fill the missing values using regression.
- To keep things simple, the remaining columns with about 1-4 missing values each are filled using median/mode imputation.

**Feature Engineering:**
- [Statistical tests](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_f_test_vs_mi.html) (F-test statistics and mutual information) are used to help feature selection. These are univariate tests so they do not take into account any interaction effects between the features, but they are simpler to understand and less computationally expensive than building models.
- Thresholds are not set, but generally columns that have one dominant value, perform poorly in the statistical tests, or give similar information to other columns (e.g. `GarageCars` vs. `GarageArea`) are all dropped. 
- The usual feature creation/transformation and ordinal/one-hot encoding are performed.

**Model Training:**
- All models are implemented using sklearn. In particular, linear models (Ridge and LinearSVR) are used as benchmarks. The main models are kernel methods (KernelRidge and SVR) and tree models (RandomForest and GradientBoostingRegressor).
- A final ensemble model, which is the average of KernelRidge, SVR, and GradientBoostingRegressor, is used for submission.

**Other things not looked at:**
- Outlier handling — my local cv results greatly improved by removing or reducing the impact of outliers, but the LB results did not improve.
- Other established feature selection/dimensionality reduction tools.
- Other ensemble techniques like stacking.

In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="darkgrid", font_scale=1.2)
import matplotlib.pyplot as plt

from scipy.stats import probplot
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV, KFold, RepeatedKFold
from sklearn.feature_selection import f_classif, mutual_info_classif, mutual_info_regression

from sklearn.svm import LinearSVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor

SEED = 0

---

# 1. Loading Datasets

In [ ]:
PATTERN = "\t| "
INCORRECT_NAMES = {
    "Bedroom": "BedroomAbvGr",
    "Kitchen": "KitchenAbvGr",
}
    
def extract_data_descr(filepath):
    data_descr = {}
    with open(filepath) as f:
        key = None
        val = []
        for line in f.readlines():
            line = line.strip()
            if line:
                line = re.split(PATTERN, line)[0]
                if key and not line.endswith(":"):
                    val.append(line)
                elif key:
                    data_descr[key] = val
                    val = []
                    key = line[:-1]
                else:
                    # for first iteration
                    key = line[:-1]

        data_descr[key] = val
      
    for old, new in INCORRECT_NAMES.items():
        data_descr[new] = data_descr.pop(old)
        
    return data_descr

In [ ]:
data_descr = extract_data_descr("/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt")

print(data_descr)

In [ ]:
X_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv").drop(columns=["Id"])
X_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv").drop(columns=["Id"])

y_train = X_train.pop("SalePrice")

---

# 2. Data Preprocessing

In [ ]:
X_train.info()

From the information of the dataframe above, many columns have missing values, and some columns may have their dtypes incorrectly labeled.

### Columns with incorrect dtypes.

Some columns have categories consisting of integers, resulting in them being recognized as numerical. We assign them back as objects. Date columns (concerning the year and month) are also usually categorical.

In [ ]:
INCORRECT_DTYPE = [
    key for key, val in data_descr.items()
    if val and np.issubdtype(X_train[key], np.number)
]
DATE_COL = [
    col for col in X_train.columns
    if any(date in col for date in ["Mo", "Yr", "Year"])
]
INCORRECT_DTYPE.extend(DATE_COL)
print(INCORRECT_DTYPE)

X_train[INCORRECT_DTYPE] = X_train[INCORRECT_DTYPE].astype(object)
X_test[INCORRECT_DTYPE] = X_test[INCORRECT_DTYPE].astype(object)

### Columns with missing values.

In [ ]:
def check_missing_values(train_df, test_df):
    print("="*40)
    print(f"(TRAIN) Number of missing values:")
    print("="*40)
    print(train_df.isna().sum()[train_df.isna().sum()>0])
    print("="*40)
    print(f"(TEST) Number of missing values:")
    print("="*40)
    print(test_df.isna().sum()[test_df.isna().sum()>0])

In [ ]:
check_missing_values(X_train, X_test)

From the data description text file, some columns have NA categories to denote that a particular element of the house does not exist (e.g. no garage). We check below that none of the data contains the string "NA", so the NA categories are most likely recognized as NaN values.

In [ ]:
print((X_train=="NA").sum().sum())
print((X_test=="NA").sum().sum())

On the other hand, it is possible that some houses have the features, but data is indeed missing. To differentiate between the two cases, we check that all related columns (of `Bsmt`, `MasVnr`, etc.) are indeed 0/NA values, indicating that the corresponding feature most likely does not exist for the house. For features with single columns (like `Alley` and `Fence`), we just fill the missing values straight.

For numerical columns, we fill using the value 0. For categorical columns, we use the string "NA".

In [ ]:
def fill_with_constant(train_df, test_df, na_col):    
    train_data = train_df.copy()
    test_data = test_df.copy()
    
    # fill all specified columns except dates
    fill_col = [
        col for col in na_col
        if all(not date in col for date in ["Yr", "Year", "Mo"])
    ]
    fill_values = train_data[fill_col].dtypes.apply(
        lambda x: 0 if np.issubdtype(x, np.number) else "NA"
    ).to_dict()
    
    train_data[~train_data[na_col].any(axis=1)] = \
    train_data[~train_data[na_col].any(axis=1)].fillna(fill_values)
    test_data[~test_data[na_col].any(axis=1)] = \
    test_data[~test_data[na_col].any(axis=1)].fillna(fill_values)
    
    return train_data, test_data

In [ ]:
HSE_FEAT = ["Bsmt", "MasVnr", "Pool", "Fireplace", "Misc", "Garage"]

RELATED_COL = {
    feat: [col for col in X_train if feat in col]
    for feat in HSE_FEAT
}

print("="*40)
print("Columns to fill missing values:")
print("="*40)
print(RELATED_COL)

In [ ]:
X_train[["Alley", "Fence"]] = X_train[["Alley", "Fence"]].fillna("NA")
X_test[["Alley", "Fence"]] = X_test[["Alley", "Fence"]].fillna("NA")

for cols in RELATED_COL.values():
    X_train, X_test = fill_with_constant(X_train, X_test, cols)
    
X_train.MasVnrType = X_train.MasVnrType.replace({"None": "NA"})
X_test.MasVnrType = X_test.MasVnrType.replace({"None": "NA"})

Instead of filling the missing values in `GarageYrBlt` column with "NA", we fix them to the values of the `YearBuilt` column. The reason is that the date columns can be used to create useful numerical features later, so this will be a possible approximation.

In [ ]:
fill_GarageYrBlt_fn = lambda x: x.YearBuilt if pd.isna(x.GarageYrBlt) else x.GarageYrBlt

X_train.GarageYrBlt = X_train.apply(fill_GarageYrBlt_fn, axis=1).astype(object)
X_test.GarageYrBlt = X_test.apply(fill_GarageYrBlt_fn, axis=1).astype(object)

In [ ]:
check_missing_values(X_train, X_test)

We still have missing values after the first round of imputation. Without any other information, we can only fill the remaining values using some form of estimation.

### Fill using estimated values.

`LotFrontage` has the largest number of missing values among the remaining columns. In general, we can expect that houses with large `LotFrontage` would also have large `LotArea`. We can verify this with the regression plot below.

In [ ]:
def regplot(X, y, method="pearson", remove_na=False, **kwargs):
    X = X.copy()
    y = y.copy()
    
    if remove_na:
        X, y = X[~( X.isna()|y.isna() )], y[~( X.isna()|y.isna() )]
    
    cc = pd.concat([X, y], axis=1).corr(method=method).values[0][1]
    mi = mutual_info_regression(X.values.reshape(-1, 1), y, random_state=SEED)[0]
    
    sns.regplot(x=X, y=y, scatter_kws=dict(edgecolors="w"), **kwargs)
    plt.title(f"Correlation Coefficient: {cc:.5f}\nMutual Information: {mi:.5f}", fontsize="large")
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(18, 4.5))

plt.subplot(121)
regplot(X_train.LotArea, X_train.LotFrontage, remove_na=True)

plt.subplot(122)
regplot(np.log1p(X_train.LotArea), np.log1p(X_train.LotFrontage), remove_na=True, color="r")
plt.xlabel("log(LotArea)")
plt.ylabel("log(LotFrontage)")

plt.show()

Using a log-log plot to reduce the effect of outlier values, we see that the log of the two columns are strongly correlated. We impute the missing values in `LotFrontage` using regression on the log values.

In [ ]:
class RegressionImputer:
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y):
        X = X.copy()
        y = y.copy()
        
        X, y = X[~( X.isna()|y.isna() )], y[~( X.isna()|y.isna() )]
        X = X.loc[y.index]
        y = np.log1p(y.values)
        X = np.log1p(X.values.reshape(-1, 1))
        self.model.fit(X, y)
        
        return self
    
    def transform(self, X, y):
        if y.isna().sum() == 0:
            return y
        
        X = X.copy()
        y = y.copy()
        
        idx = y[y.isna()].index
        X = X.loc[idx]
        X = np.log1p(X.values.reshape(-1, 1))
        y_pred = self.model.predict(X)
        y[y.isna()] = np.expm1(y_pred)
        
        return y

In [ ]:
imputer = RegressionImputer(RidgeCV())
imputer.fit(X_train.LotArea, X_train.LotFrontage)

X_train.LotFrontage = imputer.transform(X_train.LotArea, X_train.LotFrontage)
X_test.LotFrontage = imputer.transform(X_test.LotArea, X_test.LotFrontage)

To keep things simple, we fill the remaining missing values by computing the median/mode over the training data. We recall that the 0/"NA" values imputed earlier indicate that the house does not have the corresponding feature, so we need to exclude these houses for the estimations to be more representative of other houses that have such features. 

To give an example, we use `PoolQC` with 3 missing values left in the testing data. These 3 houses have `PoolArea` greater than 0, indicating that the houses have pools. However, the mode of `PoolQC` over the training data is "NA", which is a contradiction. Hence, we first remove houses without pools and then compute the mode. The same applies for other features with multiple related columns.

In [ ]:
def fill_with_estimate(train_df, test_df, related_col):
    train_data = train_df.copy()
    test_data = test_df.copy()
    
    fill_fn = lambda x: x.median() if np.issubdtype(x, np.number) else x.mode()[0]
   
    # remove houses without the corresponding feature before estimating the fill values
    fill_exclude_na = {}
    for feat, cols in related_col.items():
        hse_no_feat_idx = (train_data[cols]=="NA").sum(axis=1).astype(bool)
        fill_exclude_na.update(train_data[~hse_no_feat_idx][cols].apply(fill_fn).to_dict())
     
    # other columns
    other_col = [
        col for col in train_data.columns
        if not any(feat in col for feat in related_col.keys())
    ]
    fill_other = train_data[other_col].apply(fill_fn).to_dict()
    
    fill_values = {**fill_exclude_na, **fill_other}
    train_data = train_data.fillna(fill_values).astype(train_data.dtypes.to_dict())
    test_data = test_data.fillna(fill_values).astype(train_data.dtypes.to_dict())
    
    return train_data, test_data

In [ ]:
X_train, X_test = fill_with_estimate(X_train, X_test, related_col=RELATED_COL)

In [ ]:
check_missing_values(X_train, X_test)

---

# 3. Feature Engineering

In [ ]:
pd.concat(
    [X_train.describe().T.add_prefix("tr_"),X_test.describe().T.add_prefix("tt_")],
    axis=1,
).drop(columns=["tr_count", "tt_count"])

From the statistics of the numerical columns, we observe the following:
* Skewed distributions as indicated by the large differences between the maximum and the 75th percentile.
* Columns of different scales.

In [ ]:
pd.concat(
    [X_train.describe(include=object).T.add_prefix("tr_"), X_test.describe(include=object).T.add_prefix("tt_")],
    axis=1,
).drop(columns=["tr_count", "tt_count"])

From the statistics of the categorical columns, we note the following: 
* Many columns like `PoolQC` and `Utilities` have dominant categories that take up most of the observations.
* Many columns have a lot of unique categories.

Before moving on, we can check that the maximum `GarageYrBlt` for the testing data is 2207, which is most likely mislabeled.

In [ ]:
X_test[X_test.GarageYrBlt==X_test.GarageYrBlt.max()][DATE_COL]

Judging from the other date columns, we make a guess that the correct value is most likely 2007.

In [ ]:
X_test.loc[X_test.GarageYrBlt==2207, "GarageYrBlt"] = 2007

### Ordinal encoding.

There are a number of columns with natural ordered categories, like `OverallQual` ranging from 1 (Very Poor) to 10 (Very Excellent). The data description text file has the categories sorted in order already. We make use of the this to perform ordinal encoding to make analysis easier.

In [ ]:
# ordinal columns manually selected
ORDINAL_COL = [
    "LotShape", "LandContour", "Utilities", "LandSlope", "OverallQual",
    "OverallCond", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond",
    "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "HeatingQC", "Electrical",
    "KitchenQual", "Functional", "FireplaceQu", "GarageFinish", "GarageQual", 
    "GarageCond", "PavedDrive", "PoolQC", "Fence",
]

ORDINAL_CAT = [
    val[::-1] for key, val in data_descr.items()
    if key in ORDINAL_COL
]

print(ORDINAL_CAT)

In [ ]:
ordinal_enc = OrdinalEncoder(categories=ORDINAL_CAT)
X_train[ORDINAL_COL] = ordinal_enc.fit_transform(X_train[ORDINAL_COL].astype(str))
X_test[ORDINAL_COL] = ordinal_enc.transform(X_test[ORDINAL_COL].astype(str))

### Feature creation and transformation.

We look at creating/transforming features in the following sections to convey more useful information. To better support our analysis, we use statistical tests to determine whether or not the feature is relevant to our models. In particular, we use the Pearson's correlation coefficient/F-test statistics to measure linear correlation, and mutual information to capture nonlinear behavior. We note that these are univariate tests so they do not take into account any interaction effects between the features.

Due to large number of columns in the data, we look at each group of features separately in each section. At the beginning of each section we provide a summary of the following:
* __Features transformed__ — generally involves reducing the number of categories in each feature.

* __Features added__ — new features created from existing features and the reasoning behind it.

* __Features removed__ — feature has one dominant value, performs poorly in the univariate statistical tests, or provides similar information to other features.

In [ ]:
def multi_regplot(X, y, cols, method="pearson"):
    nrow = (len(cols)//3) + 1
    ncol = len(cols) if len(cols)<3 else 3
    fig = plt.figure(figsize=(18, 4.5*nrow))
    for i, col in enumerate(cols):
        ax = fig.add_subplot(nrow, np.max([2, ncol]), i+1)
        regplot(X[col], y, method=method, color=sns.color_palette()[i], ax=ax)
    if ncol == 1:
        fig.add_subplot(nrow, np.max([2, ncol]), i+2)
        plt.axis("off")
    plt.tight_layout()

def catplot(X, y):
    f, p = f_classif(y.values.reshape(-1, 1), X.astype(str))
    mi = mutual_info_classif(y.values.reshape(-1, 1), X.astype(str), random_state=SEED)[0]
    
    num_cat = X.nunique()
    rotation = 90 if num_cat>=7 else 0
    fig = plt.figure(figsize=(18, 4.5))
    
    plt.subplot(121)
    sns.boxplot(x=X, y=y, flierprops={"alpha": 0.5})
    plt.xticks(rotation=rotation)
    
    plt.subplot(122)
    ax = sns.countplot(x=X)
    ax.bar_label(ax.containers[0])
    plt.xticks(rotation=rotation)
    
    fig.suptitle(f"F-Statistic: {f[0]:.5f}, P-Value: {p[0]:.5f}\nMutual Information: {mi:.5f}")
    plt.tight_layout() 
    
def multi_catplot(X, y, cols):
    for col in cols:
        catplot(X[col], y)
    plt.tight_layout()

### Target column: `SalePrice`.

The target column seems to follow a log-normal distribution. We consider using the log of `SalePrice` instead so that the errors in predicting cheap and expensive houses are more even. Predicting log of `SalePrice` (which now takes both positive and negative values) also make certain models like linear regression viable. This is supported by the normal Q-Q plot below.

In [ ]:
plt.figure(figsize=(18, 4.5))

plt.subplot(121)
sns.histplot(y_train, bins=20, kde=True)
plt.title("Distribution of SalePrice", fontsize="large")

plt.subplot(122)
sns.histplot(np.log1p(y_train), bins=20, kde=True, color="orange")
plt.xlabel("log(SalePrice)")
plt.title("Distribution of log(SalePrice)", fontsize="large")

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(18, 4.5))

ax = fig.add_subplot(121)
probplot(y_train, plot=ax)
plt.title("Normal Q-Q Plot of SalePrice", fontsize="large")

ax = fig.add_subplot(122)
probplot(np.log1p(y_train), plot=ax)
plt.title("Normal Q-Q Plot of log(SalePrice)", fontsize="large")

plt.tight_layout()
plt.show()

In [ ]:
y_train = np.log1p(y_train).rename("LogSalePrice")

### Date features.

__Features transformed__
* None.

__Features added__
* (`BltToSold`, `RemodToSold`, `GrgBltToSold`) — computed by taking the difference between `YrSold` and each of (`YearBuilt`, `YearRemodAdd`, `GarageYrBlt`) respectively. We expect that newer houses or houses that are recently remodeled are more expensive. The same applies for newer garages.

* `SeasSold` — computed by grouping the months into the corresponding seasons: spring, summer, autumn, winter. We expect that house sales may have seasonal patterns, and this also helps reduce the number of categories.

__Features removed__
* `YearBuilt`, `YearRemodAdd`, `GarageYrBlt`, `MoSold`, `SeasSold`, `YrSold`.

In [ ]:
YR_COL = ["YearBuilt", "YearRemodAdd", "GarageYrBlt"]
RENAME_YR = {
    "YearBuilt": "BltToSold",
    "YearRemodAdd": "RemodToSold",
    "GarageYrBlt": "GrgBltToSold",
}

X_train[YR_COL] = (-X_train[YR_COL].sub(X_train.YrSold, axis=0)).astype(float)
X_test[YR_COL] = (-X_test[YR_COL].sub(X_test.YrSold, axis=0)).astype(float)

X_train = X_train.rename(columns=RENAME_YR)
X_test = X_test.rename(columns=RENAME_YR)

multi_regplot(X_train, y_train, cols=RENAME_YR.values())

In [ ]:
def month_to_season(mth):
    assert mth>=0 and mth<=12
    if mth==12 or mth<=2:
        return "winter"
    elif mth<=5:
        return "spring"
    elif mth<=8:
        return "summer"
    else:
        return "autumn"

In [ ]:
X_train["SeasSold"] = X_train.MoSold.apply(month_to_season)
X_test["SeasSold"] = X_test.MoSold.apply(month_to_season)

multi_catplot(X_train, y_train, cols=["MoSold", "SeasSold"])

In [ ]:
X_train = X_train.drop(columns=["MoSold", "SeasSold"], errors="ignore")
X_test = X_test.drop(columns=["MoSold", "SeasSold"], errors="ignore")

In [ ]:
multi_catplot(X_train, y_train, cols=["YrSold"])

In [ ]:
X_train = X_train.drop(columns=["YrSold"], errors="ignore")
X_test = X_test.drop(columns=["YrSold"], errors="ignore")

### Outdoor area features.

__Features transformed__
* None.

__Features added__
* `OutArea` — computed by summing (`OpenPorchSF`, `EnclosedPorch`, `3SsnPorch`, `ScreenPorch`, `WoodDeckSF`) to indicate the total outdoor area of the house.

* `NumOutType` — computed by counting how many of each (`OpenPorchSF`, `EnclosedPorch`, `3SsnPorch`, `ScreenPorch`, `WoodDeckSF`) are greater than 0. This is an alternative to `OutArea`; instead of looking at the exact area, we count the types of outdoor structures the house has.

* (`HasOpenPorch`, `HasEnclosedPorch`, `Has3SsnPorch`, `HasScreenPorch`, `HasWoodDeck`) — indicator columns for each of (`OpenPorchSF`, `EnclosedPorch`, `3SsnPorch`, `ScreenPorch`, `WoodDeckSF`) respectively.

__Features removed__
* `OpenPorchSF`, `EnclosedPorch`, `3SsnPorch`, `ScreenPorch`, `WoodDeckSF`, `Has3SsnPorch`.

In [ ]:
OUT_COL = [col for col in X_train.columns if "Porch" in col] + ["WoodDeckSF"]
print(OUT_COL)

X_train["OutArea"] = X_train[OUT_COL].sum(axis=1)
X_test["OutArea"] = X_test[OUT_COL].sum(axis=1)

multi_regplot(X_train, y_train, cols=OUT_COL+["OutArea"])

In [ ]:
X_train["NumOutType"] = (X_train[OUT_COL]>0).sum(axis=1)
X_test["NumOutType"] = (X_test[OUT_COL]>0).sum(axis=1)

multi_catplot(X_train, y_train, cols=["NumOutType"])

In [ ]:
OUT_REPLACE = {
    "OpenPorchSF": "HasOpenPorch",
    "EnclosedPorch": "HasEnclosedPorch",
    "3SsnPorch": "Has3SsnPorch",
    "ScreenPorch": "HasScreenPorch",
    "WoodDeckSF": "HasWoodDeck",
}

X_train[OUT_COL] = (X_train[OUT_COL]>0).replace({True: "Yes", False: "No"})
X_test[OUT_COL] = (X_test[OUT_COL]>0).replace({True: "Yes", False: "No"})

X_train = X_train.rename(columns=OUT_REPLACE)
X_test = X_test.rename(columns=OUT_REPLACE)

multi_catplot(X_train, y_train, cols=OUT_REPLACE.values())

In [ ]:
X_train = X_train.drop(columns=["Has3SsnPorch"], errors="ignore")
X_test = X_test.drop(columns=["Has3SsnPorch"], errors="ignore")

### Pool features.

__Features transformed__
* None.

__Features added__
* None.

__Features removed__
* `PoolArea`, `PoolQC`.

In [ ]:
multi_regplot(X_train, y_train, cols=["PoolArea"])
multi_catplot(X_train, y_train, cols=["PoolQC"])

In [ ]:
X_train = X_train.drop(columns=["PoolArea", "PoolQC"], errors="ignore")
X_test = X_test.drop(columns=["PoolArea", "PoolQC"], errors="ignore")

### Indoor area features.

It is worth noting that `TotalBsmtSF`=`BsmtFinSF1`+`BsmtFinSF2`+`BsmtUnfSF` and `GrLivArea`=`1stFlrSF`+`2ndFlrSF`+`LowQualFinSF`.

__Features transformed__
* None.

__Features added__
* (`Has2ndFlr`, `HasBsmt`) — indicator columns for each of (`2ndFlrSF`, `TotalBsmtSF`) respectively.

__Features removed__
* `BsmtFinSF2`, `LowQualFinSF`.

In [ ]:
IN_COL = [col for col in X_train if "SF" in col and not col in OUT_COL] + ["GrLivArea"]
print(IN_COL)

multi_regplot(X_train, y_train, cols=IN_COL)

In [ ]:
X_train["Has2ndFlr"] = X_train["2ndFlrSF"].apply(lambda x: "Yes" if x>0 else "No")
X_test["Has2ndFlr"] = X_test["2ndFlrSF"].apply(lambda x: "Yes" if x>0 else "No")

X_train["HasBsmt"] = X_train.TotalBsmtSF.apply(lambda x: "Yes" if x>0 else "No")
X_test["HasBsmt"] = X_test.TotalBsmtSF.apply(lambda x: "Yes" if x>0 else "No")

multi_catplot(X_train, y_train, cols=["Has2ndFlr", "HasBsmt"])

In [ ]:
X_train = X_train.drop(columns=["BsmtFinSF2", "LowQualFinSF"], errors="ignore")
X_test = X_test.drop(columns=["BsmtFinSF2", "LowQualFinSF"], errors="ignore")

### Other basement features.

__Features transformed__
* None.

__Features added__
* None.

__Features removed__
* None.

In [ ]:
BSMT_COL = [
    col for col in X_train.columns 
    if (not col in IN_COL+["HasBsmt"])
    and ("Bsmt" in col)
    and (not "Bath" in col)
]
print(BSMT_COL)

multi_catplot(X_train, y_train, BSMT_COL)

### Bathroom features.

__Features transformed__
* None.

__Features added__
* (`TtlFullBath`, `TtlHalfBath`) — computed by summing (`FullBath`, `BsmtFullBath`) and (`HalfBath`, `BsmtHalfBath`) respectively.

* `TtlBath` — computed by summing `TtlFullBath` and `TtlHalfBath`. We multiply 0.5 to `TtlHalfBath` because full bathrooms and half bathrooms are fundamentally different. This acts as an approximation.

* `HasHalfBath` — indicator column for `TtlHalfBath`. Having half bathrooms may be seen as additional feature that is good to have in a house.

* `HasBsmtBath` — indicator column computed by checking if the sum of (`BsmtFullBath`, `BsmtHalfBath`) is greater than 0. Similarly, this may be seen as an additional feature.

__Features removed__
* `BsmtFullBath`, `BsmtHalfBath`.

In [ ]:
FULLBATH_COL = ["FullBath", "BsmtFullBath"]
HALFBATH_COL = ["HalfBath", "BsmtHalfBath"]

X_train["TtlFullBath"] = X_train[FULLBATH_COL].sum(axis=1)
X_test["TtlFullBath"] = X_test[FULLBATH_COL].sum(axis=1)

X_train["TtlHalfBath"] = X_train[HALFBATH_COL].sum(axis=1)
X_test["TtlHalfBath"] = X_test[HALFBATH_COL].sum(axis=1)

X_train["TtlBath"] = X_train.TtlFullBath + 0.5*X_train.TtlHalfBath
X_test["TtlBath"] = X_test.TtlFullBath + 0.5*X_test.TtlHalfBath

BATH_COL = [col for col in X_train.columns if "Bath" in col]
print(BATH_COL)

multi_catplot(X_train, y_train, cols=BATH_COL)

In [ ]:
X_train["HasHalfBath"] = X_train[HALFBATH_COL].sum(axis=1).apply(lambda x: "Yes" if x>0 else "No")
X_test["HasHalfBath"] = X_test[HALFBATH_COL].sum(axis=1).apply(lambda x: "Yes" if x>0 else "No")

X_train["HasBsmtBath"] = X_train[["BsmtFullBath", "BsmtHalfBath"]].sum(axis=1).apply(lambda x: "Yes" if x>0 else "No")
X_test["HasBsmtBath"] = X_test[["BsmtFullBath", "BsmtHalfBath"]].sum(axis=1).apply(lambda x: "Yes" if x>0 else "No")

multi_catplot(X_train, y_train, cols=["HasHalfBath", "HasBsmtBath"])

In [ ]:
X_train = X_train.drop(columns=["BsmtFullBath", "BsmtHalfBath"], errors="ignore")
X_test = X_test.drop(columns=["BsmtFullBath", "BsmtHalfBath"], errors="ignore")

### Room features (excluding bathrooms).

__Features transformed__
* None.

__Features added__
* `OtherRms` — computed by subtracting `BedroomAbvGr` and `KitchenAbvGr` from `TotRmsAbvGrd`. Because kitchens and bedrooms are essentials in houses, may be good to know how many additional rooms there are instead.

__Features removed__
* `KitchenAbvGr`.

In [ ]:
ROOM_COL = ["TotRmsAbvGrd", "BedroomAbvGr", "KitchenAbvGr"]

X_train["OtherRms"] = X_train.TotRmsAbvGrd-X_train.BedroomAbvGr-X_train.KitchenAbvGr
X_test["OtherRms"] = X_test.TotRmsAbvGrd-X_test.BedroomAbvGr-X_test.KitchenAbvGr

multi_catplot(X_train, y_train, cols=ROOM_COL+["OtherRms"])

In [ ]:
X_train = X_train.drop(columns=["KitchenAbvGr"], errors="ignore")
X_test = X_test.drop(columns=["KitchenAbvGr"], errors="ignore")

In [ ]:
multi_catplot(X_train, y_train, cols=["KitchenQual"])

### Heating features.

__Features transformed__
* None.

__Features added__
* None.

__Features removed__
* `Heating`.

In [ ]:
HEATING_COL = ["Heating", "HeatingQC"]

multi_catplot(X_train, y_train, cols=HEATING_COL)

In [ ]:
X_train = X_train.drop(columns=["Heating"], errors="ignore")
X_test = X_test.drop(columns=["Heating"], errors="ignore")

### Fireplace features.

__Features transformed__
* None.

__Features added__
* `HasFirePlace` — indicator column for `Fireplaces`.

__Features removed__
* None.

In [ ]:
FIREPLACE_COL = ["Fireplaces", "FireplaceQu"]

X_train["HasFirePlace"] = X_train.Fireplaces.apply(lambda x: "Yes" if x else "No")
X_test["HasFirePlace"] = X_test.Fireplaces.apply(lambda x: "Yes" if x else "No")

multi_catplot(X_train, y_train, cols=FIREPLACE_COL+["HasFirePlace"])

### Garage features.

__Features transformed__
* `GarageType` — grouped some categories together.

__Features added__
* None.

__Features removed__
* `GarageCars` — removed because it gives the same information as `GarageArea`. `GarageArea` is kept as it is better at assuming continuous values.

In [ ]:
GARAGE_COL = [
    col for col in X_train.columns 
    if "Garage" in col and not col in ["GarageCars", "GarageArea"]
]
print(GARAGE_COL)

multi_catplot(X_train, y_train, cols=GARAGE_COL)

In [ ]:
GARAGETYPE_KEEP = ["Attchd", "Detchd", "BuiltIn", "NA"]

X_train.GarageType = X_train.GarageType.apply(lambda x: x if x in GARAGETYPE_KEEP else "Others")
X_test.GarageType = X_test.GarageType.apply(lambda x: x if x in GARAGETYPE_KEEP else "Others")

In [ ]:
multi_regplot(X_train, y_train, cols=["GarageCars", "GarageArea"])

In [ ]:
multi_regplot(X_train, X_train.GarageArea, cols=["GarageCars"])

In [ ]:
X_train = X_train.drop(columns=["GarageCars"], errors="ignore")
X_test = X_test.drop(columns=["GarageCars"], errors="ignore")

### Roof features.

__Features transformed__
* `RoofStyle` — grouped some categories together.

__Features added__
* None.

__Features removed__
* `RoofMatl`.

In [ ]:
ROOF_COL = ["RoofStyle", "RoofMatl"]

multi_catplot(X_train, y_train, cols=ROOF_COL)

In [ ]:
X_train.RoofStyle = X_train.RoofStyle.apply(lambda x: x if x in ["Gable", "Hip"] else "Others")
X_test.RoofStyle = X_test.RoofStyle.apply(lambda x: x if x in ["Gable", "Hip"] else "Others")

X_train = X_train.drop(columns=["RoofMatl"], errors="ignore")
X_test = X_test.drop(columns=["RoofMatl"], errors="ignore")

### Exterior covering features.

__Features transformed__
* `Exterior1st` — grouped some categories together.

* `Exterior2nd` — transformed into an indicator column by comparing against `Exterior1st`. `Exterior1st` denotes the material used for exterior covering and `Exterior2nd` indicates if more than one material is used. Since 1245 out of 1460 houses are the same, we believe that this indicates only one material is used for the 1245 houses. The remaining houses use two materials.

__Features added__
* None.

__Features removed__
* `Exterior2nd`.

In [ ]:
(X_train.Exterior1st==X_train.Exterior2nd).sum()

In [ ]:
X_train.Exterior2nd = (X_train.Exterior1st!=X_train.Exterior2nd).astype(object)
X_test.Exterior2nd = (X_test.Exterior1st!=X_test.Exterior2nd).astype(object)

In [ ]:
EXTER_COL = [col for col in X_train.columns if "Exter" in col]
print(EXTER_COL)

multi_catplot(X_train, y_train, cols=EXTER_COL)

In [ ]:
EXT1_KEEP = ["VinylSd",  "MetalSd", "Wd Sdng", "HdBoard", "Plywood"]

X_train.Exterior1st = X_train.Exterior1st.apply(lambda x: x if x in EXT1_KEEP else "Others")
X_test.Exterior1st = X_test.Exterior1st.apply(lambda x: x if x in EXT1_KEEP else "Others")

X_train = X_train.drop(columns=["Exterior2nd"], errors="ignore")
X_test = X_test.drop(columns=["Exterior2nd"], errors="ignore")

### Masonry veneer features.

__Features transformed__
* None.

__Features added__
* None.

__Features removed__
* None.

In [ ]:
multi_regplot(X_train, y_train, cols=["MasVnrArea"])
multi_catplot(X_train, y_train, cols=["MasVnrType"])

### Proximity condition features.

__Features transformed__
* `Condition1` — grouped some categories together.

* `Condition2` — transformed into an indicator column by comparing against `Condition1`. `Condition1` refers to the proximity to various conditions while `Condition2` indicates if more than one condition is present. Since 1265 out of 1460 houses are the same, we believe that this indicates that the 1265 houses only face one condition. The remaining houses face two conditions.

__Features added__
* None.

__Features removed__
* None.

In [ ]:
(X_train.Condition1==X_train.Condition2).sum()

In [ ]:
X_train.Condition2 = (X_train.Condition1!=X_train.Condition2).astype(object)
X_test.Condition2 = (X_test.Condition1!=X_test.Condition2).astype(object)

multi_catplot(X_train, y_train, cols=["Condition1", "Condition2"])

In [ ]:
COND1_REPLACE = {
    "RRNn": "RR", "RRAn": "RR",
    "PosN": "Pos", "PosA": "Pos",
    "RRNe": "RR", "RRAe": "RR",
}

X_train.Condition1 = X_train.Condition1.replace(COND1_REPLACE)
X_test.Condition1 = X_test.Condition1.replace(COND1_REPLACE)

### House lot features.

__Features transformed__
* `LotConfig` — grouped some categories together.

__Features added__
* None.

__Features removed__
* None.

In [ ]:
LOT_NUM_COL = ["LotFrontage", "LotArea"]

multi_regplot(X_train, y_train, cols=LOT_NUM_COL)

In [ ]:
LOT_COL = ["LotShape", "LotConfig"]

multi_catplot(X_train, y_train, cols=LOT_COL)

In [ ]:
LOTCONFIG_KEEP = ["Inside", "Corner", "CulDSac"]

X_train.LotConfig = X_train.LotConfig.apply(lambda x: x if x in LOTCONFIG_KEEP else "FR")
X_test.LotConfig = X_test.LotConfig.apply(lambda x: x if x in LOTCONFIG_KEEP else "FR")

### House sale features.

__Features transformed__
* (`SaleType`, `SaleCondition`) — grouped some categories together for each feature.

__Features added__
* None.

__Features removed__
* None.

In [ ]:
SALE_COL = ["SaleType", "SaleCondition"]

multi_catplot(X_train, y_train, cols=SALE_COL)

In [ ]:
SALETYPE_REPLACE = {
    "CWD": "WD", "VWD": "WD",
    "Con": "Others", "ConLw": "Others", "ConLI": "Others", "ConLD": "Others", "Oth": "Others"
}

X_train.SaleType = X_train.SaleType.replace(SALETYPE_REPLACE)
X_test.SaleType = X_test.SaleType.replace(SALETYPE_REPLACE)

SALE_COND_KEEP = ["Normal", "Partial", "Abnorml"]

X_train.SaleCondition = X_train.SaleCondition.apply(lambda x: x if x in SALE_COND_KEEP else "Others")
X_test.SaleCondition = X_test.SaleCondition.apply(lambda x: x if x in SALE_COND_KEEP else "Others")

### Misc features.

__Features transformed__
* None.

__Features added__
* None.

__Features removed__
* `MiscVal`, `MiscFeature`.

In [ ]:
multi_regplot(X_train, y_train, cols=["MiscVal"])
multi_catplot(X_train, y_train, cols=["MiscFeature"])

In [ ]:
X_train = X_train.drop(columns=["MiscVal", "MiscFeature"], errors="ignore")
X_test = X_test.drop(columns=["MiscVal", "MiscFeature"], errors="ignore")

### Other location features.

__Features transformed__
* `Alley` — grouped some categories together.

__Features added__
* None.

__Features removed__
* `Street`, `LandSlope`.

In [ ]:
OTHER_LOC_COL = ["MSZoning", "Street", "Alley", "LandContour", "LandSlope", "Neighborhood"]

multi_catplot(X_train, y_train, cols=OTHER_LOC_COL)

In [ ]:
X_train.Alley = X_train.Alley.apply(lambda x: "No" if x=="NA" else "Yes")
X_test.Alley = X_test.Alley.apply(lambda x: "No" if x=="NA" else "Yes")

X_train = X_train.drop(columns=["Street", "LandSlope"], errors="ignore")
X_test = X_test.drop(columns=["Street", "LandSlope"], errors="ignore")

### Other house features.

__Features transformed__
* (`BldgTyp`, `HouseStyle`, `OverallQual`, `OverallCond`, `CentralAir`,
    `Electrical`, `Functional`, `PavedDrive`, `Fence`, `Foundation`) — grouped some categories together for each feature.

__Features added__
* None.

__Features removed__
* `MSSubClass` — removed because it seems to be a combination of the `BldgType`, `HouseStyle` and `YearBuilt`, want to avoid having too many columns after dummy encoding later, and also not have columns with repeated information.
* `Utilities`.

In [ ]:
X_train = X_train.drop(columns=["MSSubClass"], errors="ignore")
X_test = X_test.drop(columns=["MSSubClass"], errors="ignore")

In [ ]:
OTHER_HSE_COL = [
    "BldgType", "HouseStyle", "OverallQual", "OverallCond", "CentralAir",
    "Electrical", "Functional", "PavedDrive", "Fence", "Foundation",
    "Utilities",
]

multi_catplot(X_train, y_train, cols=OTHER_HSE_COL)

In [ ]:
BLDGTYPE_REPLACE = {
    "2fmCon": "2Fam",
    "Duplex": "2Fam",
    "TwnhsE": "Twnhs",
}

X_train.BldgType = X_train.BldgType.replace(BLDGTYPE_REPLACE)
X_test.BldgType = X_test.BldgType.replace(BLDGTYPE_REPLACE)

HOUSESTYLE_REPLACE = {
    "1.5Fin": "1.5Story", "1.5Unf": "1.5Story",
    "2.5Unf": "Others", "2.5Fin": "Others",
    "SLvl": "Others", "SFoyer": "Others",
}

X_train.HouseStyle = X_train.HouseStyle.replace(HOUSESTYLE_REPLACE)
X_test.HouseStyle = X_test.HouseStyle.replace(HOUSESTYLE_REPLACE)

X_train.Electrical = X_train.Electrical.apply(lambda x: "Yes" if x==4 else "No")
X_test.Electrical = X_test.Electrical.apply(lambda x: "Yes" if x==4 else "No")

X_train.Functional = X_train.Functional.apply(lambda x: "Yes" if x==7 else "No")
X_test.Functional = X_test.Functional.apply(lambda x: "Yes" if x==7 else "No")

X_train.PavedDrive = X_train.PavedDrive.apply(lambda x: "No" if x==0 else "Yes")
X_test.PavedDrive = X_test.PavedDrive.apply(lambda x: "No" if x==0 else "Yes")

X_train.Fence = X_train.Fence.apply(lambda x: "Yes" if x else "No")
X_test.Fence = X_test.Fence.apply(lambda x: "Yes" if x else "No")

X_train.Foundation = X_train.Foundation.apply(lambda x: x if x in ["PConc", "CBlock", "BrkTil"] else "Others")
X_test.Foundation = X_test.Foundation.apply(lambda x: x if x in ["PConc", "CBlock", "BrkTil"] else "Others")

X_train = X_train.drop(columns=["Utilities"], errors="ignore")
X_test = X_test.drop(columns=["Utilities"], errors="ignore")

### Recording columns by types.

In [ ]:
NOMINAL_COL = X_train.select_dtypes(include=object).columns.to_list()
print("="*40)
print("List of nominal columns:")
print("="*40)
print(NOMINAL_COL)

ORDINAL_COL = [col for col in X_train.columns if not col in NOMINAL_COL and col in ORDINAL_COL]
print("="*40)
print("List of ordinal columns:")
print("="*40)
print(ORDINAL_COL)

NUMERICAL_COL = [col for col in X_train.columns if not col in NOMINAL_COL and not col in ORDINAL_COL]
print("="*40)
print("List of numerical columns:")
print("="*40)
print(NUMERICAL_COL)

### Dummy encoding.

We perform dummy encoding on the categorical columns. We first combine the training data and testing data together before generating the dummy columns. This ensures that all possible categories in the columns will not be left out.

In [ ]:
def dummy_encoder(train_df, test_df):    
    train_data = train_df.copy() 
    test_data = test_df.copy()
    
    # combine training and testing data to ensure all possible categories are present
    train_data["Train"] = 1
    test_data["Train"] = 0
    data = pd.concat([train_data, test_data], ignore_index=True)
    cat_data = data.select_dtypes(include=object).astype(str)
    if len(cat_data.columns) == 0:
        raise Exception("Missing categorical columns in training/testing data")
        
    # convert categorical columns to dummy columns
    dummies_df = pd.get_dummies(cat_data, drop_first=True)
    data = pd.concat([dummies_df, data.select_dtypes(exclude=object)], axis=1)
    train_data = data[data.Train==1].drop(columns=["Train"])
    test_data = data[data.Train==0].drop(columns=["Train"])
    
    # remove constant columns in training data
    const_col = train_data.nunique()[train_data.nunique()==1].index.tolist()
    train_data = train_data.drop(columns=const_col)
    test_data = test_data.drop(columns=const_col)
    
    train_data = train_data.astype(float)
    test_data = test_data.astype(float)
    
    print("Dimensions of training data from", train_df.shape, "to", train_data.shape)
    print("Dimensions of testing data from", test_df.shape, "to", test_data.shape)
    
    return train_data, test_data

In [ ]:
X_train, X_test = dummy_encoder(X_train, X_test)

### Scaling data.

Before we move on to train our regression models, we scale the data to let the models treat them with equal importance (needed for some models). It does not make sense to standardize nominal columns because there is no intrinsic ordering. Here we treat the ordinal columns as numerical, that is we assume the difference between categories 0 and 1 and the difference between categories 1 and 2 are equal. Thus, we scale only the numerical and ordinal columns.

In [ ]:
scaler = StandardScaler()

X_train[NUMERICAL_COL+ORDINAL_COL] = scaler.fit_transform(X_train[NUMERICAL_COL+ORDINAL_COL])
X_test[NUMERICAL_COL+ORDINAL_COL] = scaler.transform(X_test[NUMERICAL_COL+ORDINAL_COL])

---

# 4. Model Training

We look at the performance of the following individual models:
* __Linear models__ — Ridge and Linear SVR.

* __Kernel methods__ — Kernel Ridge and Kernel SVR.

* __Tree models__ — Random Forest and Gradient Boosting.

The linear models will be used as a benchmark against the more sophisticated models. To tune the models, we perform a simple cross-validated grid search.

In [ ]:
class ModelTuner:
    def __init__(self, models, X, y, cv, loss):
        self.init_models = models.copy()
        self.models = models.copy()
        self.X_train = X.copy()
        self.y_train = y.copy()
        self.cv = cv
        self.loss = loss
        self.train_scores = {}
        self.valid_scores = {}
        
    def tune_model(self, model_name, grid, verbose=1, n_jobs=-1):
        self.models[model_name] = self.init_models[model_name]
        search = GridSearchCV(
            self.models[model_name],
            param_grid=grid,
            scoring=self.loss,
            cv=self.cv,
            return_train_score=True,
            verbose=verbose,
            n_jobs=n_jobs,
        )
        search.fit(self.X_train, self.y_train)
        self.models[model_name] = search.best_estimator_
        
        rename_metric = {
            "mean_train_score": "mean_train_RMSE",
            "mean_test_score": "mean_valid_RMSE",
        }        
        
        cv_results = pd.DataFrame(search.cv_results_)
        columns = [f"param_{param}" for param, val in grid.items() if len(val)>1]
        columns.extend(rename_metric.keys())
        cv_results = cv_results[columns].rename(columns=rename_metric)
        cv_results[list(rename_metric.values())] = -cv_results[list(rename_metric.values())]
        cv_results = cv_results.sort_values(by="mean_valid_RMSE", ignore_index=True)

        train_score = cv_results.mean_train_RMSE.iloc[0]
        valid_score = cv_results.mean_valid_RMSE.iloc[0]
        self.train_scores[model_name] = train_score
        self.valid_scores[model_name] = valid_score
        
        print("="*40)
        print(f"Model: {search.best_estimator_}")
        print(f"Train RMSE: {train_score:.5f}")
        print(f"Valid RMSE: {valid_score:.5f}")
        print("="*40)

        return cv_results
    
    def collate_results(self):
        train_results = pd.DataFrame.from_dict(
            self.train_scores,
            orient="index",
            columns=["mean_train_RMSE"],
        )
        valid_results = pd.DataFrame.from_dict(
            self.valid_scores,
            orient="index",
            columns=["mean_valid_RMSE"],
        )
        results = pd.concat(
            [train_results, valid_results],
            axis=1,
        ).sort_values(by="mean_valid_RMSE")
        
        return results
    
    def get_models(self, model_names=None):
        if not model_names:
            model_names = self.models.keys()
            
        models = [(name, self.models[name]) for name in model_names]
        
        return models
    
    def run_cv(self, model, n_jobs=-1):
        model = self.models[model] if model in self.models else model
        scores = cross_validate(
            model,
            X=self.X_train,
            y=self.y_train,
            cv=self.cv,
            return_train_score=True,
            scoring=self.loss,
            n_jobs=n_jobs,
        )
        train_scores = scores["train_score"]
        valid_scores = scores["test_score"]
        
        print(f"Model: {model}")
        print(f"Train RMSE: {-np.mean(train_scores):.5f}")
        print(f"Valid RMSE: {-np.mean(valid_scores):.5f}")

    
    def model_predict(self, model, X):
        model = self.models[model] if model in self.models else model
        model.fit(self.X_train, self.y_train)
        y_pred = model.predict(X)
        y_pred = pd.DataFrame({
            "Id": range(1461, 1461+len(y_pred)),
            "SalePrice": np.expm1(y_pred),
        })

        return y_pred
    
    def perf_boxplot(self, add_models=None, cv=None, n_jobs=-1):
        models = self.models.copy()
        if add_models:
            for name, model in add_models:
                models[name] = model
            
        perf_df = pd.DataFrame()
        for name, model in models.items():
            valid_scores = cross_validate(
                model,
                X=self.X_train,
                y=self.y_train,
                cv=cv if cv else self.cv,
                scoring=self.loss,
                n_jobs=n_jobs,
            )["test_score"]
            
            model_perf = pd.DataFrame({"Model": name, "Validation RMSE": -valid_scores})
            perf_df = pd.concat([perf_df, model_perf])
        
        min_median = perf_df.groupby("Model").median().min()[0]
        
        plt.figure(figsize=(18, 8))
        
        sns.boxplot(
            x=perf_df["Validation RMSE"],
            y=perf_df["Model"],
            flierprops={"alpha": 0.5},
        )
        plt.axvline(x=min_median, ls="--", color="coral")
        plt.ylabel("")
        plt.title("Model Performance", fontsize="large")
        plt.tight_layout()

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)
rmse_loss = make_scorer(mean_squared_error, squared=False, greater_is_better=False)

MODELS = {
    "Ridge": Ridge(random_state=SEED),
    "Linear SVR": LinearSVR(random_state=SEED),
    "Kernel Ridge": KernelRidge(),
    "Kernel SVR": SVR(),
    "Random Forest": RandomForestRegressor(random_state=SEED),
    "Gradient Boosting": GradientBoostingRegressor(random_state=SEED),
}

tuner = ModelTuner(MODELS, X=X_train, y=y_train, cv=kfold, loss=rmse_loss)

### Ridge.

In [ ]:
RR_PARAM = {
    "alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1., 1e1],
}

rr_results = tuner.tune_model("Ridge", grid=RR_PARAM)

rr_results

### Linear SVR.

In [ ]:
SVR_PARAM = {
    "C": [1e-4, 1e-3, 1e-2, 1e-1, 1., 1e1],
    "epsilon": [0., 1e-2, 1e-1],
    "dual": [False],
    "loss": ["squared_epsilon_insensitive"],
    "tol": [1e-7],

}

svr_results = tuner.tune_model("Linear SVR", grid=SVR_PARAM)

svr_results

### Kernel Ridge.

In [ ]:
KRR_PARAM = {
    "alpha": [1e-4, 1e-3, 1e-2, 1e-1, 1., 1e1],
    "gamma": [1e-4, 1e-3, 1e-2, 1e-1],
    "kernel": ["poly", "rbf"],
}

krr_results = tuner.tune_model("Kernel Ridge", grid=KRR_PARAM)

krr_results

### Kernel SVR.

In [ ]:
KSVR_PARAM = {
    "C": [1e-4, 1e-3, 1e-2, 1e-1, 1., 1e1],
    "epsilon": [0., 1e-2, 1e-1],
    "gamma": [1e-4, 1e-3, 1e-2, 1e-1],
    "kernel": ["poly", "rbf"],
}

ksvr_results = tuner.tune_model("Kernel SVR", grid=KSVR_PARAM)

ksvr_results.head(50)

### Random Forest.

In [ ]:
RF_PARAM = {
    "max_depth": [8, 16, 32, 64],
    "n_estimators": [100, 200, 400],
    "max_features": ["sqrt"],
    "bootstrap": [False],
}

rf_results = tuner.tune_model("Random Forest", grid=RF_PARAM)

rf_results

### Gradient Boosting.

In [ ]:
GB_PARAM = {
    "max_depth": [3, 5, 10],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 4],
    "learning_rate": [1e-2, 1e-1],
    "n_estimators": [3200],
    "max_features": ["sqrt"],
}

gb_results = tuner.tune_model("Gradient Boosting", grid=GB_PARAM)

gb_results

### Ensemble Average.

In [ ]:
tuner.collate_results()

Ensemble averaging involves averaging the predictions of individual models to obtain the final prediction. Before selecting the appropriate models to include in our ensemble, we look at the model results above.
* __Bottom three models__: The bottom two are Ridge and Linear SVR, showing similar performance. It appears that there is some nonlinearity in the data which thus cannot be captured by the linear models. Random Forest performs only slightly better and shows signs of overfitting, with the train RMSE being significantly smaller than all other models.

* __Top three models__: The top three show similar performance in terms of the valid RMSE, but Gradient Boosting has much better train RMSE than both Kernel Ridge and Kernel SVR.

For our ensemble model, we choose to include only the top three models. The bottom three models appear to have relatively poor fit and may end up pulling down the performance instead of achieving the intended effect of "averaging out" the errors.

In [ ]:
models_selected = ["Kernel Ridge", "Kernel SVR", "Gradient Boosting"]
indv_models = tuner.get_models(model_names=models_selected)
ensemble = VotingRegressor(indv_models)

tuner.run_cv(ensemble)

### Summary.

We look at the overall validation performance of all the models. To have a better picture, we repeat the cross-validation process using RepeatedKFold to get more samples of the valid RMSE for each model. The ensemble model has the smallest median value of the validation RMSE as indicated by the red vertical dashed line.

In [ ]:
rkfold = RepeatedKFold(n_splits=10, n_repeats=10, random_state=SEED)

tuner.perf_boxplot(add_models=[("Ensemble Average", ensemble)], cv=rkfold)

---

# 5. Submission

We use the ensemble average as our final model.

In [ ]:
submission = tuner.model_predict(ensemble, X_test)

submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)

---